# Bits to Symbols

In [45]:
# Imports & Basics

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna as sn

# For plotting
%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

# for performance measurements
import time

# For the implementation of the Keras models
from tensorflow.keras import Model

from sionna.utils.misc import hard_decisions
from sionna.utils.metrics import compute_ber, compute_ser

CODERATE = 0.5
n = 16
k = int(n*CODERATE)

NUM_BITS_PER_SYMBOL = 4 # QPSK
BLOCK_LENGTH = k
BATCH_SIZE = 2 # How many examples are processed by Sionna in parallel
EBN0_DB_MIN = -10.0 # Minimum value of Eb/N0 [dB] for simulations
EBN0_DB_MAX = 10.0 # Maximum value of Eb/N0 [dB] for simulations

# Binary source
binary_source = sn.utils.BinarySource()

# Constellation
constellation = sn.mapping.Constellation("qam", NUM_BITS_PER_SYMBOL)

bits = binary_source([BATCH_SIZE,BLOCK_LENGTH])
print('bits =\n',bits)

new_shape = [-1] + bits.shape[1:-1].as_list() + \
           [int(bits.shape[-1] / NUM_BITS_PER_SYMBOL),
            NUM_BITS_PER_SYMBOL]
print('new_shape =\n',new_shape)

bits_reshaped = tf.cast(tf.reshape(bits, new_shape), tf.int32)
print('bits_reshaped =\n',bits_reshaped)

binary_base = 2**tf.constant(
                        range(NUM_BITS_PER_SYMBOL-1,-1,-1))
print('binary_base =\n',binary_base)

int_rep = tf.reduce_sum(bits_reshaped * binary_base, axis=-1)
print('int_rep =\n',int_rep)

x = tf.gather(constellation.points, int_rep, axis=0)
print(x)

bits =
 tf.Tensor(
[[1. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1.]], shape=(2, 8), dtype=float32)
new_shape =
 [-1, 2, 4]
bits_reshaped =
 tf.Tensor(
[[[1 1 1 0]
  [0 1 0 1]]

 [[1 0 1 1]
  [1 1 1 1]]], shape=(2, 2, 4), dtype=int32)
binary_base =
 tf.Tensor([8 4 2 1], shape=(4,), dtype=int32)
int_rep =
 tf.Tensor(
[[14  5]
 [11 15]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[-0.9486833-0.3162278j  0.3162278-0.9486833j]
 [-0.9486833+0.9486833j -0.9486833-0.9486833j]], shape=(2, 2), dtype=complex64)


# EBN0_DB to EBN0

In [50]:
# Imports & Basics

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna as sn

# For plotting
%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

# for performance measurements
import time

# For the implementation of the Keras models
from tensorflow.keras import Model

from sionna.utils.misc import hard_decisions
from sionna.utils.metrics import compute_ber, compute_ser

NUM_BITS_PER_SYMBOL = 4 # QPSK
BLOCK_LENGTH = 8
BATCH_SIZE = 2
EBN0_DB_MIN = -3.0
EBN0_DB_MAX = 3.0

for EBN0_DB in np.linspace(EBN0_DB_MIN,EBN0_DB_MAX,20):
    print('EBN0_DB =',EBN0_DB)
    no = sn.utils.ebnodb2no(ebno_db=EBN0_DB,
                            num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
                            coderate=1.0)
    print('no =',no)


EBN0_DB = -3.0
no = tf.Tensor(0.4988156, shape=(), dtype=float32)
EBN0_DB = -2.6842105263157894
no = tf.Tensor(0.46383238, shape=(), dtype=float32)
EBN0_DB = -2.3684210526315788
no = tf.Tensor(0.43130267, shape=(), dtype=float32)
EBN0_DB = -2.0526315789473686
no = tf.Tensor(0.4010543, shape=(), dtype=float32)
EBN0_DB = -1.736842105263158
no = tf.Tensor(0.37292734, shape=(), dtype=float32)
EBN0_DB = -1.4210526315789473
no = tf.Tensor(0.346773, shape=(), dtype=float32)
EBN0_DB = -1.105263157894737
no = tf.Tensor(0.32245293, shape=(), dtype=float32)
EBN0_DB = -0.7894736842105265
no = tf.Tensor(0.29983848, shape=(), dtype=float32)
EBN0_DB = -0.47368421052631593
no = tf.Tensor(0.27881005, shape=(), dtype=float32)
EBN0_DB = -0.1578947368421053
no = tf.Tensor(0.2592564, shape=(), dtype=float32)
EBN0_DB = 0.1578947368421053
no = tf.Tensor(0.2410741, shape=(), dtype=float32)
EBN0_DB = 0.4736842105263155
no = tf.Tensor(0.22416694, shape=(), dtype=float32)
EBN0_DB = 0.7894736842105261
no = tf.Ten

# sionna.channel.AWGN

In [95]:
# Imports & Basics

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna as sn

# For plotting
%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

# for performance measurements
import time

# For the implementation of the Keras models
from tensorflow.keras import Model

from sionna.utils import expand_to_rank, complex_normal

NUM_BITS_PER_SYMBOL = 4 # QPSK
BLOCK_LENGTH = 8
BATCH_SIZE = 2
EBN0_DB = -10

# Binary source
binary_source = sn.utils.BinarySource()

bits = binary_source([BATCH_SIZE,BLOCK_LENGTH])
print('bits =\n',bits)

no = sn.utils.ebnodb2no(ebno_db=EBN0_DB,
                        num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
                        coderate=1.0)
print('no =',no)

# Constellation
constellation = sn.mapping.Constellation("qam", NUM_BITS_PER_SYMBOL)
#constellation.show(figsize=(7,7));

# Mapper and Demapper
mapper = sn.mapping.Mapper(constellation=constellation)

x = mapper(bits)
print('x =\n',x)
 
# Create tensors of real-valued Gaussian noise for each complex dim.
noise = complex_normal(tf.shape(x), dtype=x.dtype)
print('noise =\n',noise)

# Add extra dimensions for broadcasting
no = expand_to_rank(no, tf.rank(x), axis=-1)
print('no =',no)

# Apply variance scaling
real_dtype = tf.dtypes.as_dtype(tf.complex64).real_dtype
no = tf.cast(no, real_dtype)
print('no =',no)
noise *= tf.cast(tf.sqrt(no), noise.dtype)
print('noise =\n',noise)

# Add noise to input
y = x + noise
#print('y = x + noise =\n',y)


bits =
 tf.Tensor(
[[1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 1. 1.]], shape=(2, 8), dtype=float32)
no = tf.Tensor(2.5, shape=(), dtype=float32)
x =
 tf.Tensor(
[[-0.9486833-0.9486833j -0.3162278+0.3162278j]
 [ 0.9486833-0.9486833j  0.9486833-0.9486833j]], shape=(2, 2), dtype=complex64)
noise =
 tf.Tensor(
[[-0.2457148 -0.4940561j  -0.2502328 +0.0766773j ]
 [ 0.6659922 -0.00514933j  0.40934482-0.22090244j]], shape=(2, 2), dtype=complex64)
no = tf.Tensor([[2.5]], shape=(1, 1), dtype=float32)
no = tf.Tensor([[2.5]], shape=(1, 1), dtype=float32)
noise =
 tf.Tensor(
[[-0.3885092 -0.7811713j  -0.39565277+0.12123746j]
 [ 1.0530262 -0.00814181j  0.647231  -0.34927744j]], shape=(2, 2), dtype=complex64)


# logits2llrs

In [87]:
# Imports & Basics

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna as sn

# For plotting
%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

# for performance measurements
import time

# For the implementation of the Keras models
from tensorflow.keras import Model

def SymbolLogits2LLRs(method, num_bits_per_symbol, hard_out=False, with_prior=False, dtype=tf.float32, **kwargs):
    # pylint: disable=line-too-long
    r"""
    SymbolLogits2LLRs(method, num_bits_per_symbol, hard_out=False, with_prior=False, dtype=tf.float32, **kwargs)

    Computes log-likelihood ratios (LLRs) or hard-decisions on bits
    from a tensor of logits (i.e., unnormalized log-probabilities) on constellation points.
    If the flag ``with_prior`` is set, prior knowledge on the bits is assumed to be available.

    Parameters
    ----------
    method : One of ["app", "maxlog"], str
        The method used for computing the LLRs.

    num_bits_per_symbol : int
        The number of bits per constellation symbol, e.g., 4 for QAM16.

    hard_out : bool
        If `True`, the layer provides hard-decided bits instead of soft-values.
        Defaults to `False`.

    with_prior : bool
        If `True`, it is assumed that prior knowledge on the bits is available.
        This prior information is given as LLRs as an additional input to the layer.
        Defaults to `False`.

    dtype : One of [tf.float32, tf.float64] tf.DType (dtype)
        The dtype for the input and output.
        Defaults to `tf.float32`.

    Input
    -----
    logits or (logits, prior):
        Tuple:

    logits : [...,n, num_points], tf.float
        Logits on constellation points.

    prior : [num_bits_per_symbol] or [...n, num_bits_per_symbol], tf.float
        Prior for every bit as LLRs.
        It can be provided either as a tensor of shape `[num_bits_per_symbol]`
        for the entire input batch, or as a tensor that is "broadcastable"
        to `[..., n, num_bits_per_symbol]`.
        Only required if the ``with_prior`` flag is set.

    Output
    ------
    : [...,n, num_bits_per_symbol], tf.float
        LLRs or hard-decisions for every bit.

    Note
    ----
    With the "app" method, the LLR for the :math:`i\text{th}` bit
    is computed according to

    .. math::
        LLR(i) = \ln\left(\frac{\Pr\left(b_i=1\lvert \mathbf{z},\mathbf{p}\right)}{\Pr\left(b_i=0\lvert \mathbf{z},\mathbf{p}\right)}\right) =\ln\left(\frac{
                \sum_{c\in\mathcal{C}_{i,1}} \Pr\left(c\lvert\mathbf{p}\right)
                e^{z_c}
                }{
                \sum_{c\in\mathcal{C}_{i,0}} \Pr\left(c\lvert\mathbf{p}\right)
                e^{z_c}
                }\right)

    where :math:`\mathcal{C}_{i,1}` and :math:`\mathcal{C}_{i,0}` are the
    sets of :math:`2^K` constellation points for which the :math:`i\text{th}` bit is
    equal to 1 and 0, respectively. :math:`\mathbf{z} = \left[z_{c_0},\dots,z_{c_{2^K-1}}\right]` is the vector of logits on the constellation points, :math:`\mathbf{p} = \left[p_0,\dots,p_{K-1}\right]`
    is the vector of LLRs that serves as prior knowledge on the :math:`K` bits that are mapped to
    a constellation point and is set to :math:`\mathbf{0}` if no prior knowledge is assumed to be available,
    and :math:`\Pr(c\lvert\mathbf{p})` is the prior probability on the constellation symbol :math:`c`:

    .. math::
        \Pr\left(c\lvert\mathbf{p}\right) = \prod_{k=0}^{K-1} \Pr\left(b_k = \ell(c)_k \lvert\mathbf{p} \right)
        = \prod_{k=0}^{K-1} \text{sigmoid}\left(p_k \ell(c)_k\right)

    where :math:`\ell(c)_k` is the :math:`k^{th}` bit label of :math:`c`, where 0 is
    replaced by -1.
    The definition of the LLR has been
    chosen such that it is equivalent with that of logits. This is
    different from many textbooks in communications, where the LLR is
    defined as :math:`LLR(i) = \ln\left(\frac{\Pr\left(b_i=0\lvert y\right)}{\Pr\left(b_i=1\lvert y\right)}\right)`.

    With the "maxlog" method, LLRs for the :math:`i\text{th}` bit
    are approximated like

    .. math::
        \begin{align}
            LLR(i) &\approx\ln\left(\frac{
                \max_{c\in\mathcal{C}_{i,1}} \Pr\left(c\lvert\mathbf{p}\right)
                    e^{z_c}
                }{
                \max_{c\in\mathcal{C}_{i,0}} \Pr\left(c\lvert\mathbf{p}\right)
                    e^{z_c}
                }\right)
                .
        \end{align}
    """
    def __init__(self,
                 method,
                 num_bits_per_symbol,
                 hard_out=False,
                 with_prior=False,
                 dtype=tf.float32,
                 **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert method in ("app","maxlog"), "Unknown demapping method"
        self._method = method
        self._hard_out = hard_out
        self._num_bits_per_symbol = num_bits_per_symbol
        self._with_prior = with_prior
        num_points = int(2**num_bits_per_symbol)

        # Array composed of binary representations of all symbols indices
        a = np.zeros([num_points, num_bits_per_symbol])
        for i in range(0, num_points):
            a[i,:] = np.array(list(np.binary_repr(i, num_bits_per_symbol)),
                              dtype=np.int16)

        # Compute symbol indices for which the bits are 0 or 1
        c0 = np.zeros([int(num_points/2), num_bits_per_symbol])
        c1 = np.zeros([int(num_points/2), num_bits_per_symbol])
        for i in range(num_bits_per_symbol-1,-1,-1):
            c0[:,i] = np.where(a[:,i]==0)[0]
            c1[:,i] = np.where(a[:,i]==1)[0]
        self._c0 = tf.constant(c0, dtype=tf.int32) # Symbols with ith bit=0
        self._c1 = tf.constant(c1, dtype=tf.int32) # Symbols with ith bit=1

        if with_prior:
            # Array of labels from {-1, 1} of all symbols
            # [num_points, num_bits_per_symbol]
            a = 2*a-1
            self._a = tf.constant(a, dtype=dtype)

        # Determine the reduce function for LLR computation
        if self._method == "app":
            self._reduce = tf.reduce_logsumexp
        else:
            self._reduce = tf.reduce_max

    @property
    def num_bits_per_symbol(self):
        return self._num_bits_per_symbol

    def call(self, inputs):
        if self._with_prior:
            logits, prior = inputs
        else:
            logits = inputs

        # Compute exponents
        exponents = logits

        # Gather exponents for all bits
        # shape [...,n,num_points/2,num_bits_per_symbol]
        exp0 = tf.gather(exponents, self._c0, axis=-1, batch_dims=0)
        exp1 = tf.gather(exponents, self._c1, axis=-1, batch_dims=0)

        # Process the prior information
        if self._with_prior:
            # Expanding `prior` such that it is broadcastable with
            # shape [..., n or 1, 1, num_bits_per_symbol]
            prior = sn.utils.expand_to_rank(prior, tf.rank(logits), axis=0)
            prior = tf.expand_dims(prior, axis=-2)

            # Expand the symbol labeling to be broadcastable with prior
            # shape [..., 1, num_points, num_bits_per_symbol]
            a = sn.utils.expand_to_rank(self._a, tf.rank(prior), axis=0)

            # Compute the prior probabilities on symbols exponents
            # shape [..., n or 1, num_points]
            exp_ps = tf.reduce_sum(tf.math.log_sigmoid(a*prior), axis=-1)

            # Gather prior probability symbol for all bits
            # shape [..., n or 1, num_points/2, num_bits_per_symbol]
            exp_ps0 = tf.gather(exp_ps, self._c0, axis=-1)
            exp_ps1 = tf.gather(exp_ps, self._c1, axis=-1)

        # Compute LLRs using the definition log( Pr(b=1)/Pr(b=0) )
        # shape [..., n, num_bits_per_symbol]
        if self._with_prior:
            llr = self._reduce(exp_ps1 + exp1, axis=-2)\
                    - self._reduce(exp_ps0 + exp0, axis=-2)
        else:
            llr = self._reduce(exp1, axis=-2) - self._reduce(exp0, axis=-2)

        if self._hard_out:
            return sn.utils.hard_decisions(llr)
        else:
            return llr

NUM_BITS_PER_SYMBOL = 4 # QPSK
BLOCK_LENGTH = 8
BATCH_SIZE = 2
EBN0_DB = -10

# Binary source
binary_source = sn.utils.BinarySource()

# Constellation
constellation = sn.mapping.Constellation("qam", NUM_BITS_PER_SYMBOL)
#constellation.show(figsize=(7,7));

# Mapper and Demapper
mapper = sn.mapping.Mapper(constellation=constellation)

# AWGN channel
awgn_channel = sn.channel.AWGN()

bits = binary_source([BATCH_SIZE,BLOCK_LENGTH])
#print('bits =\n',bits)

no = sn.utils.ebnodb2no(ebno_db=EBN0_DB,
                        num_bits_per_symbol=NUM_BITS_PER_SYMBOL,
                        coderate=1.0)
#print('no =',no)

x = mapper(bits)
#print('x =\n',x)

y = awgn_channel([x, no])

# Reshape constellation points to [1,...1,num_points]
points_shape = [1]*y.shape.rank + constellation.points.shape
points = tf.reshape(constellation.points, points_shape)

# Compute squared distances from y to all points
# shape [...,n,num_points]
squared_dist = tf.pow(tf.abs(tf.expand_dims(y, axis=-1) - points), 2)

# Add a dummy dimension for broadcasting. This is not needed when no
# is a scalar, but also does not do any harm.
no = tf.expand_dims(no, axis=-1)

# Compute exponents
exponents = -squared_dist/no
#print('exponents =\n',exponents)

logits2llrs = SymbolLogits2LLRs("app",NUM_BITS_PER_SYMBOL)
print('logits2llrs =\n',logits2llrs)

# if self._with_prior:
#     llr = self._logits2llrs([exponents, prior])
# else:
llr = logits2llrs(exponents)

# Reshape LLRs to [...,n*num_bits_per_symbol]
out_shape = tf.concat([tf.shape(y)[:-1],
                        [y.shape[-1] * \
                        constellation.num_bits_per_symbol]], 0)
llr_reshaped = tf.reshape(llr, out_shape)

# return llr_reshaped

logits2llrs =
 None


TypeError: 'NoneType' object is not callable

# bits2symbol

In [32]:
# Imports & Basics

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna as sn

# For plotting
%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

# for performance measurements
import time

# For the implementation of the Keras models
from tensorflow.keras import Model

from sionna.utils.misc import hard_decisions
from sionna.utils.metrics import compute_ber, compute_ser

def bits2symbol(bits,num_bits_per_symbol):
    new_shape = [-1] + bits.shape[1:-1].as_list() + \
            [int(bits.shape[-1] / NUM_BITS_PER_SYMBOL),
                NUM_BITS_PER_SYMBOL]
    #print('new_shape =',new_shape)
    #print('bits.shape[1:-1]',bits.shape[1:-1])
    print('bits.shape[-1]=',bits.shape[-1])

    symbols = tf.cast(tf.reshape(bits, new_shape), tf.int32)
    print('symbols =',symbols)
    return symbols


CODERATE = 0.5
n = 16
k = int(n*CODERATE)

NUM_BITS_PER_SYMBOL = 4 # QPSK
BLOCK_LENGTH = k
BATCH_SIZE = 2 # How many examples are processed by Sionna in parallel
EBN0_DB_MIN = -10.0 # Minimum value of Eb/N0 [dB] for simulations
EBN0_DB_MAX = 10.0 # Maximum value of Eb/N0 [dB] for simulations

# Binary source
binary_source = sn.utils.BinarySource()

# Constellation
constellation = sn.mapping.Constellation("qam", NUM_BITS_PER_SYMBOL)

#bits = binary_source([BATCH_SIZE,BLOCK_LENGTH])
bits = binary_source([2,3,4,5,6])
print('bits =',bits)
print('bits.ndim =',bits.ndim)

symbols = bits2symbol(bits,NUM_BITS_PER_SYMBOL)
print('symbols =',symbols)

bits = tf.Tensor(
[[[[[0. 0. 1. 1. 1. 0.]
    [1. 0. 0. 0. 1. 1.]
    [0. 0. 1. 0. 1. 0.]
    [1. 0. 0. 1. 1. 1.]
    [0. 1. 0. 1. 1. 0.]]

   [[0. 0. 0. 1. 1. 0.]
    [0. 0. 1. 0. 1. 1.]
    [0. 0. 0. 0. 1. 0.]
    [0. 1. 0. 0. 0. 0.]
    [1. 0. 1. 0. 0. 1.]]

   [[0. 0. 0. 1. 0. 0.]
    [1. 1. 1. 1. 1. 1.]
    [1. 0. 1. 1. 0. 1.]
    [1. 1. 0. 0. 1. 1.]
    [0. 0. 1. 1. 1. 0.]]

   [[0. 0. 1. 0. 0. 1.]
    [0. 1. 0. 0. 0. 0.]
    [0. 0. 0. 1. 0. 0.]
    [1. 1. 1. 1. 0. 0.]
    [0. 0. 0. 1. 0. 0.]]]


  [[[0. 1. 1. 0. 1. 0.]
    [1. 0. 0. 0. 1. 1.]
    [1. 1. 1. 0. 1. 1.]
    [1. 1. 0. 0. 1. 1.]
    [1. 0. 1. 1. 0. 0.]]

   [[0. 1. 1. 0. 0. 0.]
    [1. 0. 0. 1. 1. 0.]
    [0. 0. 0. 1. 1. 1.]
    [0. 1. 1. 0. 1. 0.]
    [0. 1. 1. 1. 1. 0.]]

   [[1. 0. 0. 1. 0. 1.]
    [0. 1. 0. 0. 0. 1.]
    [1. 0. 0. 1. 0. 1.]
    [1. 0. 1. 0. 0. 0.]
    [1. 0. 1. 0. 0. 0.]]

   [[1. 1. 1. 0. 0. 0.]
    [0. 0. 0. 0. 0. 0.]
    [1. 1. 1. 0. 0. 0.]
    [1. 1. 1. 1. 1. 1.]
    [0. 1. 0. 1. 0. 1.]]]


  